In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
sns.set(style='ticks', palette='RdBu')
#sns.set(style='ticks', palette='Set2')
from functools import reduce
import datetime
import pytz


## Cleaning

In [2]:
# paste data from bi

# order = pd.read_excel(r"C:\Users\user\Documents\Alex\Order Efficiency\February 23\Book1.xlsx")


order = pd.read_csv(r"C:\Users\user\Downloads\cda-export (56).csv", delimiter = ';')


### cut off matrix()
matrix = pd.read_excel(r"C:\Users\user\Documents\Alex\Order Efficiency\cut off matrix\cut off matrix.xlsx")



In [3]:
order.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2
1,Surfacing,SSF-SPQC,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:08:00,2023-03-25 13:01:00,173
2,Surfacing,SSF-SPQC,Surfacing Lens with Branch Frame,993575,236101144,2023-03-24 17:40:00,2023-03-25 11:51:00,191


In [4]:
order.to_clipboard()

In [5]:
order['dept'].unique()

array(['Control', 'Surfacing', 'Lens Store', 'Main Store', 'PreQuality',
       'Packaging', 'FinalQC', 'Designer'], dtype=object)

In [6]:
order.drop(order[order['dept']=='Surfacing'].index, inplace=True)
order.drop(order[order['dept']=='PreQuality'].index, inplace=True)
order.drop(order[order['dept']=='FinalQC'].index, inplace=True)

In [7]:
order['dept'].unique()

array(['Control', 'Lens Store', 'Main Store', 'Packaging', 'Designer'],
      dtype=object)

In [8]:
import datetime

In [9]:
order['finish']

0       2023-03-25 14:08:00
34      2023-03-25 10:08:00
35      2023-03-25 13:59:00
36      2023-03-25 15:38:00
37      2023-03-25 10:14:00
               ...         
2941    2023-03-25 15:40:00
2942    2023-03-25 13:00:00
2943    2023-03-25 12:32:00
2944    2023-03-25 14:16:00
2945    2023-03-25 10:38:00
Name: finish, Length: 1761, dtype: object

In [10]:
order['finish'] = order['finish'].astype(str)

In [11]:
order[['Day','Time']] = order['finish'].str.split(' ', expand = True)
order.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,14:08:00
34,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,10:08:00
35,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,13:59:00


In [12]:
order['Time'] = pd.to_datetime(order['Time'], format = '%H:%M:%S')

In [13]:
order['Hour'] = order['Time'].dt.strftime('%H')
order.head(5)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14
34,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10
35,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13
36,Control,BSC-SSF,Surfacing Lens with HQ Frame,992230,23210305,2023-03-25 15:34:00,2023-03-25 15:38:00,4,2023-03-25,1900-01-01 15:38:00,15
37,Control,BSC-SSF,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:12:00,2023-03-25 10:14:00,2,2023-03-25,1900-01-01 10:14:00,10


### Cut off matrix

In [14]:
matrix.head(5)

,Order Criteria,Main Store,Designer Store,Lens Store,Control Room,Packaging
0,Branch Frame and HQ Lens Glazed at Branch,0,0,9,9,10
1,Contact lens from HQ,10,10,0,9,10
2,Frame from HQ and Lens from HQ,10,10,9,9,10
3,Glazing from HQ,0,0,9,9,10
4,Order with New Frame from Branch and Lens from HQ,0,0,9,9,10


In [15]:
order1 = pd.merge(order, matrix, on='Order Criteria', how= 'left')
order1.head(5)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0
3,Control,BSC-SSF,Surfacing Lens with HQ Frame,992230,23210305,2023-03-25 15:34:00,2023-03-25 15:38:00,4,2023-03-25,1900-01-01 15:38:00,15,10.0,10.0,9.0,9.0,10.0
4,Control,BSC-SSF,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:12:00,2023-03-25 10:14:00,2,2023-03-25,1900-01-01 10:14:00,10,0.0,0.0,9.0,9.0,10.0


In [16]:
matrix.columns

Index(['Order Criteria', 'Main Store', 'Designer Store', 'Lens Store',
       'Control Room', 'Packaging'],
      dtype='object')

In [17]:
order1['cut off']= np.where(order1['dept']=='Control',order1['Control Room'],(np.where(order1['dept']=='Designer',order1['Designer Store'],(np.where(order1['dept']=='Main Store',order1['Main Store'],(np.where(order1['dept']=='Lens Store',order1['Lens Store'],(np.where(order1['dept']=='Packaging',order1['Packaging'],10)))))))))
order1.head(5)                                

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0
3,Control,BSC-SSF,Surfacing Lens with HQ Frame,992230,23210305,2023-03-25 15:34:00,2023-03-25 15:38:00,4,2023-03-25,1900-01-01 15:38:00,15,10.0,10.0,9.0,9.0,10.0,9.0
4,Control,BSC-SSF,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:12:00,2023-03-25 10:14:00,2,2023-03-25,1900-01-01 10:14:00,10,0.0,0.0,9.0,9.0,10.0,9.0


In [18]:
order1.to_clipboard()

In [19]:
order1['Delay'] = np.where(order1['Time Min']>order1['cut off'],1,0)
order1.head(5)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0,0
3,Control,BSC-SSF,Surfacing Lens with HQ Frame,992230,23210305,2023-03-25 15:34:00,2023-03-25 15:38:00,4,2023-03-25,1900-01-01 15:38:00,15,10.0,10.0,9.0,9.0,10.0,9.0,0
4,Control,BSC-SSF,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:12:00,2023-03-25 10:14:00,2,2023-03-25,1900-01-01 10:14:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0


In [20]:
order =order1
order['Delay'].unique()

array([0, 1])

### Criteria

In [21]:
criteria = order

In [22]:
criteria.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0,0


In [23]:
criteria1 = criteria[criteria['Delay']== 1]


In [24]:
criteria1['Delay'].unique()

array([1])

In [25]:
pivotcriteria = pd.pivot_table(criteria1, index='Order Criteria', columns='dept', values='Doc Entry', aggfunc='count', margins=True, margins_name='Grand Total' )
pivotcriteria

dept,Control,Designer,Lens Store,Main Store,Packaging,Grand Total
Order Criteria,,,,,,
Branch Frame and HQ Lens Glazed at Branch,NaN,NaN,NaN,NaN,1.0,1
Contact lens from HQ,NaN,NaN,NaN,NaN,2.0,2
Frame from HQ and Lens from HQ,7.0,6.0,NaN,2.0,7.0,22
Order with New Frame from Branch and Lens from HQ,2.0,NaN,1.0,NaN,1.0,4
Order with PF from Branch and Lens from HQ,2.0,NaN,1.0,NaN,8.0,11
Overseas Lens with Branch Frame,2.0,NaN,NaN,NaN,2.0,4
Overseas Lens with HQ Frame,1.0,1.0,NaN,NaN,3.0,5
Overseas Lens with PF,NaN,NaN,NaN,NaN,1.0,1
Overseas Lens with PF to Follow,NaN,NaN,NaN,NaN,1.0,1


In [26]:
criteria_table=pivotcriteria

## Control

### Efficiency

In [27]:
control = order[order['dept']== 'Control']

In [28]:
control['dept'].unique()

array(['Control'], dtype=object)

In [29]:
control.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0,0


In [30]:

controlpivot1 = pd.pivot_table(control, index = ['Hour'], values=['Doc No'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
controlpivot2 = pd.pivot_table(control, index=['Hour'], values=['Time Min'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
controlpivot3 = pd.pivot_table(control, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [31]:
controlpivot4= pd.merge(controlpivot1,controlpivot2, on=['Hour'], how = 'left')
controlpivot4


,Doc No,Time Min
Hour,,
09,31,2.741935
10,38,2.868421
11,85,5.576471
12,136,4.808824
13,167,7.131737
14,114,6.078947
15,79,4.278481
16,37,0.297297
17,3,0.000000


In [32]:
controlpivot = pd.merge(controlpivot4,controlpivot3, on=['Hour'], how = 'left')
controlpivot

,Doc No,Time Min,Delay
Hour,,,
09,31,2.741935,0
10,38,2.868421,0
11,85,5.576471,0
12,136,4.808824,0
13,167,7.131737,14
14,114,6.078947,1
15,79,4.278481,0
16,37,0.297297,0
17,3,0.000000,0


In [33]:
controlpivot['% of Efficiency'] = (controlpivot['Doc No'] - controlpivot['Delay'])/controlpivot['Doc No']

controlpivot['Time Min'] =controlpivot['Time Min'].round(2)
controlpivot['% of Efficiency'] =controlpivot['% of Efficiency'].map('{:.2%}'.format)
controlpivot

,Doc No,Time Min,Delay,% of Efficiency
Hour,,,,
09,31,2.74,0,100.00%
10,38,2.87,0,100.00%
11,85,5.58,0,100.00%
12,136,4.81,0,100.00%
13,167,7.13,14,91.62%
14,114,6.08,1,99.12%
15,79,4.28,0,100.00%
16,37,0.30,0,100.00%
17,3,0.00,0,100.00%


In [34]:
controlpivot = controlpivot.drop(columns={})
controlpivot

,Doc No,Time Min,Delay,% of Efficiency
Hour,,,,
09,31,2.74,0,100.00%
10,38,2.87,0,100.00%
11,85,5.58,0,100.00%
12,136,4.81,0,100.00%
13,167,7.13,14,91.62%
14,114,6.08,1,99.12%
15,79,4.28,0,100.00%
16,37,0.30,0,100.00%
17,3,0.00,0,100.00%


In [35]:
controlpivot = np.transpose(controlpivot)
controlpivot

Hour,09,10,11,12,13,14,15,16,17,Total
Doc No,31,38,85,136,167,114,79,37,3,690
Time Min,2.74,2.87,5.58,4.81,7.13,6.08,4.28,0.3,0.0,5.15
Delay,0,0,0,0,14,1,0,0,0,15
% of Efficiency,100.00%,100.00%,100.00%,100.00%,91.62%,99.12%,100.00%,100.00%,100.00%,97.83%


### Delays

In [36]:
control.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0,0


In [37]:
control1 = control

In [38]:
control1.drop(control1[control1['Delay'] == 0].index, inplace=True)
control1.head(3)

C:\Users\user\AppData\Local\Temp\ipykernel_9756\1265887728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control1.drop(control1[control1['Delay'] == 0].index, inplace=True)


,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
16,Control,BSC-SSF,Surfacing Lens with PF,1002262,23211510,2023-03-25 13:23:00,2023-03-25 13:33:00,10,2023-03-25,1900-01-01 13:33:00,13,0.0,0.0,9.0,9.0,10.0,9.0,1
341,Control,SC-SPQC,Overseas Lens with HQ Frame,992531,23210340,2023-03-25 13:12:00,2023-03-25 13:33:00,21,2023-03-25,1900-01-01 13:33:00,13,10.0,10.0,0.0,18.0,10.0,18.0,1
501,Control,SC-SPQC,Frame from HQ and Lens from HQ,1001656,23211413,2023-03-25 13:32:00,2023-03-25 13:44:00,12,2023-03-25,1900-01-01 13:44:00,13,10.0,10.0,9.0,9.0,10.0,9.0,1


In [39]:
controldelay = pd.pivot_table(control1, index=['Hour','Doc No'], values= 'Time Min', aggfunc= 'mean',fill_value=0)
controldelay

Time Min
Hour Doc No             
13   23107212         11
     23107230         11
     23210340         21
     23211413         12
     23211510         10
     23211533         14
     23700649         21
     231601022        21
     231901455        11
     234001247        11
     235100946        14
     235502633        10
     236200974        11
     236400859        14
14   234900413        10

### Cutoff

In [40]:
control2 = order
control2.head(4)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0,0
3,Control,BSC-SSF,Surfacing Lens with HQ Frame,992230,23210305,2023-03-25 15:34:00,2023-03-25 15:38:00,4,2023-03-25,1900-01-01 15:38:00,15,10.0,10.0,9.0,9.0,10.0,9.0,0


In [41]:
control2 = control2[control2['dept']=='Control']
control2['dept'].unique()

array(['Control'], dtype=object)

In [42]:
control2['15 min'] = control2['Time Min'].apply(lambda x : 1 if x >15 else 0)
control2['12 min'] = control2['Time Min'].apply(lambda x : 1 if x >12 else 0)
control2['10 min'] = control2['Time Min'].apply(lambda x : 1 if x >10 else 0)
control2['7 min'] = control2['Time Min'].apply(lambda x : 1 if x >7 else 0)
control2.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_9756\639867209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control2['15 min'] = control2['Time Min'].apply(lambda x : 1 if x >15 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\639867209.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control2['12 min'] = control2['Time Min'].apply(lambda x : 1 if x >12 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\639867209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,...,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay,15 min,12 min,10 min,7 min
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,...,10.0,9.0,9.0,10.0,9.0,0,0,0,0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,...,0.0,9.0,9.0,10.0,9.0,0,0,0,0,0


In [43]:
cuttoff1 = pd.pivot_table(control2,index='dept', values =['15 min'], aggfunc='sum', fill_value=0)
cuttoff2 = pd.pivot_table(control2,index='dept', values =['12 min'], aggfunc='sum', fill_value=0)
cuttoff3 = pd.pivot_table(control2,index='dept', values =['10 min'], aggfunc='sum', fill_value=0)
cuttoff4 = pd.pivot_table(control2,index='dept', values =['7 min'], aggfunc='sum', fill_value=0)
cutt = pd.merge(cuttoff1,cuttoff2, on='dept')
cutt = pd.merge(cutt,cuttoff3, on='dept')
cutt = pd.merge(cutt,cuttoff4, on='dept')
cuttcontrol = cutt
cuttcontrol

,15 min,12 min,10 min,7 min
dept,,,,
Control,4,22,37,141


## Designer

### Efficiency

In [44]:
Designer = order[order['dept']== 'Designer']

In [45]:
Designer['dept'].unique()

array(['Designer'], dtype=object)

In [46]:
Designer.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
939,Designer,SO-OV,Overseas Lens with HQ Frame,1001814,237000590,2023-03-25 15:03:00,2023-03-25 15:09:00,6,2023-03-25,1900-01-01 15:09:00,15,10.0,10.0,0.0,18.0,10.0,10.0,0
944,Designer,SO-SL,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:00:00,2023-03-25 14:04:00,4,2023-03-25,1900-01-01 14:04:00,14,10.0,10.0,9.0,9.0,10.0,10.0,0
948,Designer,SO-OV,Overseas Lens with HQ Frame,1001249,231101838,2023-03-24 18:08:00,2023-03-25 09:13:00,13,2023-03-25,1900-01-01 09:13:00,09,10.0,10.0,0.0,18.0,10.0,10.0,1


In [47]:

Designerlpivot1 = pd.pivot_table(Designer, index = ['Hour'], values=['Doc No'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
Designerpivot2 = pd.pivot_table(Designer, index=['Hour'], values=['Time Min'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
Designerpivot3 = pd.pivot_table(Designer, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [48]:
Designerpivot4= pd.merge(Designerlpivot1,Designerpivot2, on=['Hour'], how = 'left')
Designerpivot4


,Doc No,Time Min
Hour,,
09,6,11.333333
10,4,3.500000
11,7,5.857143
12,5,1.600000
13,14,6.214286
14,8,4.000000
15,6,3.666667
17,2,0.000000
Total,52,5.230769


In [49]:
Designerpivot = pd.merge(Designerpivot4,Designerpivot3, on=['Hour'], how = 'left')
Designerpivot

,Doc No,Time Min,Delay
Hour,,,
09,6,11.333333,5
10,4,3.500000,0
11,7,5.857143,1
12,5,1.600000,0
13,14,6.214286,2
14,8,4.000000,0
15,6,3.666667,0
17,2,0.000000,0
Total,52,5.230769,8


In [50]:
Designerpivot['% of Efficiency'] = (Designerpivot['Doc No'] - Designerpivot['Delay'])/Designerpivot['Doc No']

Designerpivot['Time Min'] =Designerpivot['Time Min'].round(2)
Designerpivot['% of Efficiency'] =Designerpivot['% of Efficiency'].map('{:.2%}'.format)
Designerpivot

,Doc No,Time Min,Delay,% of Efficiency
Hour,,,,
09,6,11.33,5,16.67%
10,4,3.50,0,100.00%
11,7,5.86,1,85.71%
12,5,1.60,0,100.00%
13,14,6.21,2,85.71%
14,8,4.00,0,100.00%
15,6,3.67,0,100.00%
17,2,0.00,0,100.00%
Total,52,5.23,8,84.62%


In [51]:
Designerpivot = np.transpose(Designerpivot)
Designerpivot

Hour,09,10,11,12,13,14,15,17,Total
Doc No,6,4,7,5,14,8,6,2,52
Time Min,11.33,3.5,5.86,1.6,6.21,4.0,3.67,0.0,5.23
Delay,5,0,1,0,2,0,0,0,8
% of Efficiency,16.67%,100.00%,85.71%,100.00%,85.71%,100.00%,100.00%,100.00%,84.62%


### Delay

In [52]:
Designer.head(2)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
939,Designer,SO-OV,Overseas Lens with HQ Frame,1001814,237000590,2023-03-25 15:03:00,2023-03-25 15:09:00,6,2023-03-25,1900-01-01 15:09:00,15,10.0,10.0,0.0,18.0,10.0,10.0,0
944,Designer,SO-SL,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:00:00,2023-03-25 14:04:00,4,2023-03-25,1900-01-01 14:04:00,14,10.0,10.0,9.0,9.0,10.0,10.0,0


In [53]:
Designer1 = Designer

In [54]:
Designer1.drop(Designer[Designer1['Delay'] == 0].index ,inplace =True)
Designer.shape

C:\Users\user\AppData\Local\Temp\ipykernel_9756\2653956933.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Designer1.drop(Designer[Designer1['Delay'] == 0].index ,inplace =True)


(8, 18)

In [55]:
Designerdelay = pd.pivot_table(Designer1, index=['Hour','Doc No'], values = 'Time Min', aggfunc = 'mean',fill_value=0)
Designerdelay

Time Min
Hour Doc No             
09   23211480         13
     23604174         13
     231101838        13
     236000677        13
     236700291        13
11   23211405         11
13   231901455        11
     234001247        13

### Cutoff

In [56]:
designer2 = order
designer2.head(4)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
0,Control,SC-SPQC,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:06:00,2023-03-25 14:08:00,2,2023-03-25,1900-01-01 14:08:00,14,10.0,10.0,9.0,9.0,10.0,9.0,0
1,Control,BSC-SSF,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:04:00,2023-03-25 10:08:00,4,2023-03-25,1900-01-01 10:08:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
2,Control,BSC-SSF,Surfacing Lens with HQ Frame,994355,23210574,2023-03-25 13:58:00,2023-03-25 13:59:00,1,2023-03-25,1900-01-01 13:59:00,13,10.0,10.0,9.0,9.0,10.0,9.0,0
3,Control,BSC-SSF,Surfacing Lens with HQ Frame,992230,23210305,2023-03-25 15:34:00,2023-03-25 15:38:00,4,2023-03-25,1900-01-01 15:38:00,15,10.0,10.0,9.0,9.0,10.0,9.0,0


In [57]:
designer2 = designer2[designer2['dept']=='Designer']
designer2['dept'].unique()

array(['Designer'], dtype=object)

In [58]:
designer2['15 min'] = designer2['Time Min'].apply(lambda x : 1 if x >15 else 0)
designer2['12 min'] = designer2['Time Min'].apply(lambda x : 1 if x >12 else 0)
designer2['8 min'] = designer2['Time Min'].apply(lambda x : 1 if x >8 else 0)
designer2['6 min'] = designer2['Time Min'].apply(lambda x : 1 if x >6 else 0)
designer2.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_9756\1996152161.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  designer2['15 min'] = designer2['Time Min'].apply(lambda x : 1 if x >15 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\1996152161.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  designer2['12 min'] = designer2['Time Min'].apply(lambda x : 1 if x >12 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\1996152161.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,...,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay,15 min,12 min,8 min,6 min
939,Designer,SO-OV,Overseas Lens with HQ Frame,1001814,237000590,2023-03-25 15:03:00,2023-03-25 15:09:00,6,2023-03-25,1900-01-01 15:09:00,...,10.0,0.0,18.0,10.0,10.0,0,0,0,0,0
944,Designer,SO-SL,Frame from HQ and Lens from HQ,1000468,23107088,2023-03-25 14:00:00,2023-03-25 14:04:00,4,2023-03-25,1900-01-01 14:04:00,...,10.0,9.0,9.0,10.0,10.0,0,0,0,0,0


In [59]:
cuttoff1 = pd.pivot_table(designer2,index='dept', values =['15 min'], aggfunc='sum', fill_value=0)
cuttoff2 = pd.pivot_table(designer2,index='dept', values =['12 min'], aggfunc='sum', fill_value=0)
cuttoff3 = pd.pivot_table(designer2,index='dept', values =['8 min'], aggfunc='sum', fill_value=0)
cuttoff4 = pd.pivot_table(designer2,index='dept', values =['6 min'], aggfunc='sum', fill_value=0)
cutt = pd.merge(cuttoff1,cuttoff2, on='dept')
cutt = pd.merge(cutt,cuttoff3, on='dept')
cutt = pd.merge(cutt,cuttoff4, on='dept')
cuttdesigner = cutt
cuttdesigner

,15 min,12 min,8 min,6 min
dept,,,,
Designer,0,6,11,15


## Main Store

### Efficiency

In [60]:
MainStore = order[order['dept']== 'Main Store']

In [61]:
MainStore['dept'].unique()

array(['Main Store'], dtype=object)

In [62]:
MainStore.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
67,Main Store,SO-SC,Contact lens from HQ,1002230,23107204,2023-03-25 11:13:00,2023-03-25 11:16:00,3,2023-03-25,1900-01-01 11:16:00,11,10.0,10.0,0.0,9.0,10.0,10.0,0
68,Main Store,SO-SC,Contact lens from HQ,1002386,232700775,2023-03-25 12:29:00,2023-03-25 12:34:00,5,2023-03-25,1900-01-01 12:34:00,12,10.0,10.0,0.0,9.0,10.0,10.0,0
69,Main Store,SO-SC,Contact lens from HQ,1002402,231402170,2023-03-25 12:30:00,2023-03-25 12:34:00,4,2023-03-25,1900-01-01 12:34:00,12,10.0,10.0,0.0,9.0,10.0,10.0,0


In [63]:

MainStorepivot1 = pd.pivot_table(MainStore, index = ['Hour'], values=['Doc No'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
MainStorepivot2 = pd.pivot_table(MainStore, index=['Hour'], values=['Time Min'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
MainStorepivot3 = pd.pivot_table(MainStore, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [64]:
MainStorepivot4= pd.merge(MainStorepivot1,MainStorepivot2, on=['Hour'], how = 'left')
MainStorepivot4


,Doc No,Time Min
Hour,,
09,12,2.000000
10,6,2.833333
11,18,2.444444
12,25,16.680000
13,39,4.307692
14,37,4.702703
15,38,4.526316
16,17,0.058824
Total,192,5.296875


In [65]:
MainStorepivot = pd.merge(MainStorepivot4,MainStorepivot3, on=['Hour'], how = 'left')
MainStorepivot

,Doc No,Time Min,Delay
Hour,,,
09,12,2.000000,0
10,6,2.833333,0
11,18,2.444444,0
12,25,16.680000,1
13,39,4.307692,0
14,37,4.702703,0
15,38,4.526316,1
16,17,0.058824,0
Total,192,5.296875,2


In [66]:
MainStorepivot['% of Efficiency'] = (MainStorepivot['Doc No'] - MainStorepivot['Delay'])/MainStorepivot['Doc No']

MainStorepivot['Time Min'] =MainStorepivot['Time Min'].round(2)
MainStorepivot['% of Efficiency'] =MainStorepivot['% of Efficiency'].map('{:.2%}'.format)
MainStorepivot

,Doc No,Time Min,Delay,% of Efficiency
Hour,,,,
09,12,2.00,0,100.00%
10,6,2.83,0,100.00%
11,18,2.44,0,100.00%
12,25,16.68,1,96.00%
13,39,4.31,0,100.00%
14,37,4.70,0,100.00%
15,38,4.53,1,97.37%
16,17,0.06,0,100.00%
Total,192,5.30,2,98.96%


In [67]:
MainStorepivot = np.transpose(MainStorepivot)
MainStorepivot

Hour,09,10,11,12,13,14,15,16,Total
Doc No,12,6,18,25,39,37,38,17,192
Time Min,2.0,2.83,2.44,16.68,4.31,4.7,4.53,0.06,5.3
Delay,0,0,0,1,0,0,1,0,2
% of Efficiency,100.00%,100.00%,100.00%,96.00%,100.00%,100.00%,97.37%,100.00%,98.96%


### Delay

In [68]:
MainStore1 = MainStore

In [69]:
MainStore1.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
67,Main Store,SO-SC,Contact lens from HQ,1002230,23107204,2023-03-25 11:13:00,2023-03-25 11:16:00,3,2023-03-25,1900-01-01 11:16:00,11,10.0,10.0,0.0,9.0,10.0,10.0,0
68,Main Store,SO-SC,Contact lens from HQ,1002386,232700775,2023-03-25 12:29:00,2023-03-25 12:34:00,5,2023-03-25,1900-01-01 12:34:00,12,10.0,10.0,0.0,9.0,10.0,10.0,0
69,Main Store,SO-SC,Contact lens from HQ,1002402,231402170,2023-03-25 12:30:00,2023-03-25 12:34:00,4,2023-03-25,1900-01-01 12:34:00,12,10.0,10.0,0.0,9.0,10.0,10.0,0


In [70]:
MainStore1.drop(MainStore1[MainStore1['Delay']==0].index, inplace = True)
MainStore1.shape

C:\Users\user\AppData\Local\Temp\ipykernel_9756\1121525000.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MainStore1.drop(MainStore1[MainStore1['Delay']==0].index, inplace = True)


(2, 18)

In [71]:
MainStore1delay = pd.pivot_table(MainStore1, index=['Hour','Doc No'], values='Time Min', aggfunc= 'mean',fill_value=0)
MainStore1delay

,,Time Min
Hour,Doc No,
12,23211437,332
15,23107234,20


### Cutoff

In [72]:
MainStore2 = order
MainStore2['dept'].unique()

array(['Control', 'Lens Store', 'Main Store', 'Packaging', 'Designer'],
      dtype=object)

In [73]:
MainStore2 = MainStore2[MainStore2['dept']=='Main Store']
MainStore2['dept'].unique()

array(['Main Store'], dtype=object)

In [74]:
MainStore2['15 min'] = MainStore2['Time Min'].apply(lambda x : 1 if x >15 else 0)
MainStore2['12 min'] = MainStore2['Time Min'].apply(lambda x : 1 if x >12 else 0)
MainStore2['8 min'] = MainStore2['Time Min'].apply(lambda x : 1 if x >8 else 0)
MainStore2['6 min'] = MainStore2['Time Min'].apply(lambda x : 1 if x >6 else 0)
MainStore2.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_9756\2741039344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MainStore2['15 min'] = MainStore2['Time Min'].apply(lambda x : 1 if x >15 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\2741039344.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MainStore2['12 min'] = MainStore2['Time Min'].apply(lambda x : 1 if x >12 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\2741039344.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,...,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay,15 min,12 min,8 min,6 min
67,Main Store,SO-SC,Contact lens from HQ,1002230,23107204,2023-03-25 11:13:00,2023-03-25 11:16:00,3,2023-03-25,1900-01-01 11:16:00,...,10.0,0.0,9.0,10.0,10.0,0,0,0,0,0
68,Main Store,SO-SC,Contact lens from HQ,1002386,232700775,2023-03-25 12:29:00,2023-03-25 12:34:00,5,2023-03-25,1900-01-01 12:34:00,...,10.0,0.0,9.0,10.0,10.0,0,0,0,0,0


In [75]:
cuttoff1 = pd.pivot_table(MainStore2,index='dept', values =['15 min'], aggfunc='sum', fill_value=0)
cuttoff2 = pd.pivot_table(MainStore2,index='dept', values =['12 min'], aggfunc='sum', fill_value=0)
cuttoff3 = pd.pivot_table(MainStore2,index='dept', values =['8 min'], aggfunc='sum', fill_value=0)
cuttoff4 = pd.pivot_table(MainStore2,index='dept', values =['6 min'], aggfunc='sum', fill_value=0)
cutt = pd.merge(cuttoff1,cuttoff2, on='dept')
cutt = pd.merge(cutt,cuttoff3, on='dept')
cutt = pd.merge(cutt,cuttoff4, on='dept')
cuttMainStore2 = cutt
cuttMainStore2

,15 min,12 min,8 min,6 min
dept,,,,
Main Store,2,2,5,23


## Packaging

### Efficiency

In [76]:
Packaging = order[order['dept']== 'Packaging']

In [77]:
Packaging['dept'].unique()

array(['Packaging'], dtype=object)

In [78]:
Packaging.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
90,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1001314,234203235,2023-03-25 15:41:00,2023-03-25 15:45:00,4,2023-03-25,1900-01-01 15:45:00,15,0.0,0.0,9.0,9.0,10.0,10.0,0
91,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1001804,232502883,2023-03-25 15:41:00,2023-03-25 15:45:00,4,2023-03-25,1900-01-01 15:45:00,15,0.0,0.0,9.0,9.0,10.0,10.0,0
92,Packaging,SP-GBSTB,Branch Frame and HQ Lens Glazed at Branch,1002049,231901446,2023-03-25 09:27:00,2023-03-25 09:31:00,4,2023-03-25,1900-01-01 09:31:00,09,0.0,0.0,9.0,9.0,10.0,10.0,0


In [79]:

Packagingpivot1 = pd.pivot_table(Packaging, index = ['Hour'], values=['Doc No'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
Packagingpivot2 = pd.pivot_table(Packaging, index=['Hour'], values=['Time Min'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
Packagingpivot3 = pd.pivot_table(Packaging, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [80]:
Packagingpivot4= pd.merge(Packagingpivot1,Packagingpivot2, on=['Hour'], how = 'left')
Packagingpivot4


,Doc No,Time Min
Hour,,
09,16,3.812500
10,43,3.186047
11,29,2.310345
12,55,4.563636
13,64,4.671875
14,83,7.168675
15,98,6.795918
16,79,0.253165
17,26,0.000000


In [81]:
Packagingpivot = pd.merge(Packagingpivot4,Packagingpivot3, on=['Hour'], how = 'left')
Packagingpivot

,Doc No,Time Min,Delay
Hour,,,
09,16,3.812500,0
10,43,3.186047,0
11,29,2.310345,1
12,55,4.563636,1
13,64,4.671875,0
14,83,7.168675,24
15,98,6.795918,2
16,79,0.253165,0
17,26,0.000000,0


In [82]:
Packagingpivot['% of Efficiency'] = (Packagingpivot['Doc No'] - Packagingpivot['Delay'])/Packagingpivot['Doc No']

Packagingpivot['Time Min'] =Packagingpivot['Time Min'].round(2)
Packagingpivot['% of Efficiency'] =Packagingpivot['% of Efficiency'].map('{:.2%}'.format)
Packagingpivot

,Doc No,Time Min,Delay,% of Efficiency
Hour,,,,
09,16,3.81,0,100.00%
10,43,3.19,0,100.00%
11,29,2.31,1,96.55%
12,55,4.56,1,98.18%
13,64,4.67,0,100.00%
14,83,7.17,24,71.08%
15,98,6.80,2,97.96%
16,79,0.25,0,100.00%
17,26,0.00,0,100.00%


In [83]:
Packagingpivot = np.transpose(Packagingpivot)
Packagingpivot

Hour,09,10,11,12,13,14,15,16,17,Total
Doc No,16,43,29,55,64,83,98,79,26,493
Time Min,3.81,3.19,2.31,4.56,4.67,7.17,6.8,0.25,0.0,4.25
Delay,0,0,1,1,0,24,2,0,0,28
% of Efficiency,100.00%,100.00%,96.55%,98.18%,100.00%,71.08%,97.96%,100.00%,100.00%,94.32%


### Delay

In [84]:
Packaging1=Packaging

In [85]:
Packaging1.head(4)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
90,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1001314,234203235,2023-03-25 15:41:00,2023-03-25 15:45:00,4,2023-03-25,1900-01-01 15:45:00,15,0.0,0.0,9.0,9.0,10.0,10.0,0
91,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1001804,232502883,2023-03-25 15:41:00,2023-03-25 15:45:00,4,2023-03-25,1900-01-01 15:45:00,15,0.0,0.0,9.0,9.0,10.0,10.0,0
92,Packaging,SP-GBSTB,Branch Frame and HQ Lens Glazed at Branch,1002049,231901446,2023-03-25 09:27:00,2023-03-25 09:31:00,4,2023-03-25,1900-01-01 09:31:00,09,0.0,0.0,9.0,9.0,10.0,10.0,0
93,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1002097,234203255,2023-03-25 09:27:00,2023-03-25 09:31:00,4,2023-03-25,1900-01-01 09:31:00,09,0.0,0.0,9.0,9.0,10.0,10.0,0


In [86]:
Packaging1.drop(Packaging1[Packaging1['Delay']==0].index, inplace = True)
Packaging.shape

C:\Users\user\AppData\Local\Temp\ipykernel_9756\321544454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Packaging1.drop(Packaging1[Packaging1['Delay']==0].index, inplace = True)


(28, 18)

In [87]:
Packagingdelay = pd.pivot_table(Packaging1, index= ['Hour','Doc No'], values = 'Time Min', aggfunc='mean',fill_value=0,margins=True,margins_name='Total')
Packagingdelay

Time Min
Hour  Doc No              
11    23107204   27.000000
12    226109550  18.000000
14    23106393   14.000000
      23107213   13.000000
      23107214   17.000000
      23107216   13.000000
      23107230   14.000000
      23107231   31.000000
      23209034   14.000000
      23209286   14.000000
      23209932   14.000000
      23211332   16.000000
      231501276  11.000000
      231601172  21.000000
      232301062  14.000000
      232700770  12.000000
      232802019  11.000000
      232900454  12.000000
      234100943  16.000000
      234202861  14.000000
      234301778  11.000000
      235502627  13.000000
      235502629  15.000000
      235700419  12.000000
      236101104  14.000000
      236800828  16.000000
15    23211529   82.000000
      236600580  66.000000
Total            19.464286

In [88]:
Packagingdelay = pd.pivot_table(Packaging1, index=['Hour','Doc No'], values='Time Min', aggfunc= 'mean',fill_value=0)
Packagingdelay

Time Min
Hour Doc No             
11   23107204         27
12   226109550        18
14   23106393         14
     23107213         13
     23107214         17
     23107216         13
     23107230         14
     23107231         31
     23209034         14
     23209286         14
     23209932         14
     23211332         16
     231501276        11
     231601172        21
     232301062        14
     232700770        12
     232802019        11
     232900454        12
     234100943        16
     234202861        14
     234301778        11
     235502627        13
     235502629        15
     235700419        12
     236101104        14
     236800828        16
15   23211529         82
     236600580        66

### Cutoff

In [89]:
packaging2 = order
packaging2['dept'].unique()

array(['Control', 'Lens Store', 'Main Store', 'Packaging', 'Designer'],
      dtype=object)

In [90]:
packaging2 = packaging2[packaging2['dept']=='Packaging']
packaging2['dept'].unique()

array(['Packaging'], dtype=object)

In [91]:
packaging2['15 min'] = packaging2['Time Min'].apply(lambda x : 1 if x >15 else 0)
packaging2['12 min'] = packaging2['Time Min'].apply(lambda x : 1 if x >12 else 0)
packaging2['8 min'] = packaging2['Time Min'].apply(lambda x : 1 if x >8 else 0)
packaging2['6 min'] = packaging2['Time Min'].apply(lambda x : 1 if x >6 else 0)
packaging2.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_9756\4127047498.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  packaging2['15 min'] = packaging2['Time Min'].apply(lambda x : 1 if x >15 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\4127047498.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  packaging2['12 min'] = packaging2['Time Min'].apply(lambda x : 1 if x >12 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\4127047498.py:3: SettingWithCopyWarning: 
A value is trying to be set on a cop

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,...,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay,15 min,12 min,8 min,6 min
90,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1001314,234203235,2023-03-25 15:41:00,2023-03-25 15:45:00,4,2023-03-25,1900-01-01 15:45:00,...,0.0,9.0,9.0,10.0,10.0,0,0,0,0,0
91,Packaging,SP-GBSTB,PF and HQ Lens Glazed at Branch,1001804,232502883,2023-03-25 15:41:00,2023-03-25 15:45:00,4,2023-03-25,1900-01-01 15:45:00,...,0.0,9.0,9.0,10.0,10.0,0,0,0,0,0


In [92]:
cuttoff1 = pd.pivot_table(packaging2,index='dept', values =['15 min'], aggfunc='sum', fill_value=0)
cuttoff2 = pd.pivot_table(packaging2,index='dept', values =['12 min'], aggfunc='sum', fill_value=0)
cuttoff3 = pd.pivot_table(packaging2,index='dept', values =['8 min'], aggfunc='sum', fill_value=0)
cuttoff4 = pd.pivot_table(packaging2,index='dept', values =['6 min'], aggfunc='sum', fill_value=0)
cutt = pd.merge(cuttoff1,cuttoff2, on='dept')
cutt = pd.merge(cutt,cuttoff3, on='dept')
cutt = pd.merge(cutt,cuttoff4, on='dept')
cuttpackaging2 = cutt
cuttpackaging2

,15 min,12 min,8 min,6 min
dept,,,,
Packaging,10,22,45,96


## Lens Store

### Efficiency

In [93]:
LensStore = order[order['dept']== 'Lens Store']

In [94]:
LensStore['dept'].unique()

array(['Lens Store'], dtype=object)

In [95]:
LensStore.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
34,Lens Store,OP-ISB,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:01:00,2023-03-25 10:04:00,3,2023-03-25,1900-01-01 10:04:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
35,Lens Store,OP-ISB,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:04:00,2023-03-25 10:11:00,7,2023-03-25,1900-01-01 10:11:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
36,Lens Store,OP-ISB,Surfacing Lens with Branch Frame,1000815,234801095,2023-03-25 11:57:00,2023-03-25 12:05:00,8,2023-03-25,1900-01-01 12:05:00,12,0.0,0.0,9.0,9.0,10.0,9.0,0


In [96]:

LensStorepivot1 = pd.pivot_table(LensStore, index = ['Hour'], values=['Doc No'], aggfunc='count', fill_value=0, margins=True, margins_name='Total')
LensStorepivot2 = pd.pivot_table(LensStore, index=['Hour'], values=['Time Min'], aggfunc = 'mean', fill_value=0, margins=True, margins_name='Total')
LensStorepivot3 = pd.pivot_table(LensStore, index=['Hour'], values=['Delay'], aggfunc = 'sum', fill_value=0, margins=True, margins_name='Total')


In [97]:
LensStorepivot4= pd.merge(LensStorepivot1,LensStorepivot2, on=['Hour'], how = 'left')
LensStorepivot4


,Doc No,Time Min
Hour,,
09,30,3.700000
10,25,4.120000
11,27,3.407407
12,47,3.936170
13,66,3.106061
14,50,3.840000
15,56,4.160714
16,31,0.322581
17,2,0.000000


In [98]:
LensStorepivot = pd.merge(LensStorepivot4,LensStorepivot3, on=['Hour'], how = 'left')
LensStorepivot

,Doc No,Time Min,Delay
Hour,,,
09,30,3.700000,0
10,25,4.120000,0
11,27,3.407407,0
12,47,3.936170,0
13,66,3.106061,0
14,50,3.840000,0
15,56,4.160714,4
16,31,0.322581,0
17,2,0.000000,0


In [99]:
LensStorepivot['% of Efficiency'] = (LensStorepivot['Doc No'] - LensStorepivot['Delay'])/LensStorepivot['Doc No']

LensStorepivot['Time Min'] =LensStorepivot['Time Min'].round(2)
LensStorepivot['% of Efficiency'] =LensStorepivot['% of Efficiency'].map('{:.2%}'.format)
LensStorepivot

,Doc No,Time Min,Delay,% of Efficiency
Hour,,,,
09,30,3.70,0,100.00%
10,25,4.12,0,100.00%
11,27,3.41,0,100.00%
12,47,3.94,0,100.00%
13,66,3.11,0,100.00%
14,50,3.84,0,100.00%
15,56,4.16,4,92.86%
16,31,0.32,0,100.00%
17,2,0.00,0,100.00%


In [100]:
LensStorepivot = np.transpose(LensStorepivot)
LensStorepivot

Hour,09,10,11,12,13,14,15,16,17,Total
Doc No,30,25,27,47,66,50,56,31,2,334
Time Min,3.7,4.12,3.41,3.94,3.11,3.84,4.16,0.32,0.0,3.39
Delay,0,0,0,0,0,0,4,0,0,4
% of Efficiency,100.00%,100.00%,100.00%,100.00%,100.00%,100.00%,92.86%,100.00%,100.00%,98.80%


### Delay

In [101]:
LensStore1 = LensStore

In [102]:
LensStore1.head(3)

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,Hour,Main Store,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay
34,Lens Store,OP-ISB,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:01:00,2023-03-25 10:04:00,3,2023-03-25,1900-01-01 10:04:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
35,Lens Store,OP-ISB,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:04:00,2023-03-25 10:11:00,7,2023-03-25,1900-01-01 10:11:00,10,0.0,0.0,9.0,9.0,10.0,9.0,0
36,Lens Store,OP-ISB,Surfacing Lens with Branch Frame,1000815,234801095,2023-03-25 11:57:00,2023-03-25 12:05:00,8,2023-03-25,1900-01-01 12:05:00,12,0.0,0.0,9.0,9.0,10.0,9.0,0


In [103]:
LensStore1.drop(LensStore1[LensStore1['Delay']==0].index, inplace = True)
LensStore1.shape

C:\Users\user\AppData\Local\Temp\ipykernel_9756\1408256614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LensStore1.drop(LensStore1[LensStore1['Delay']==0].index, inplace = True)


(4, 18)

In [104]:
LensStore1delay = pd.pivot_table(LensStore1, index=['Hour','Doc No'], values='Time Min',aggfunc='mean',fill_value=0)
LensStore1delay

Time Min
Hour Doc No             
15   232802026        11
     232802038        10
     232802040        10
     234001246        13

### Cutoff

In [105]:
lens2 = order
lens2['dept'].unique()

array(['Control', 'Lens Store', 'Main Store', 'Packaging', 'Designer'],
      dtype=object)

In [106]:
lens2 = lens2[lens2['dept']=='Lens Store']
lens2['dept'].unique()

array(['Lens Store'], dtype=object)

In [107]:
lens2['15 min'] = lens2['Time Min'].apply(lambda x : 1 if x >15 else 0)
lens2['12 min'] = lens2['Time Min'].apply(lambda x : 1 if x >12 else 0)
lens2['8 min'] = lens2['Time Min'].apply(lambda x : 1 if x >8 else 0)
lens2['6 min'] = lens2['Time Min'].apply(lambda x : 1 if x >6 else 0)
lens2.head(2)

C:\Users\user\AppData\Local\Temp\ipykernel_9756\1285314021.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lens2['15 min'] = lens2['Time Min'].apply(lambda x : 1 if x >15 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\1285314021.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lens2['12 min'] = lens2['Time Min'].apply(lambda x : 1 if x >12 else 0)
C:\Users\user\AppData\Local\Temp\ipykernel_9756\1285314021.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

,dept,status,Order Criteria,Doc Entry,Doc No,start,finish,Time Min,Day,Time,...,Designer Store,Lens Store,Control Room,Packaging,cut off,Delay,15 min,12 min,8 min,6 min
34,Lens Store,OP-ISB,Surfacing Lens with PF,1000906,234400610,2023-03-25 10:01:00,2023-03-25 10:04:00,3,2023-03-25,1900-01-01 10:04:00,...,0.0,9.0,9.0,10.0,9.0,0,0,0,0,0
35,Lens Store,OP-ISB,Surfacing Lens with Branch Frame,999963,233300709,2023-03-25 10:04:00,2023-03-25 10:11:00,7,2023-03-25,1900-01-01 10:11:00,...,0.0,9.0,9.0,10.0,9.0,0,0,0,0,1


In [108]:
cuttoff1 = pd.pivot_table(lens2,index='dept', values =['15 min'], aggfunc='sum', fill_value=0)
cuttoff2 = pd.pivot_table(lens2,index='dept', values =['12 min'], aggfunc='sum', fill_value=0)
cuttoff3 = pd.pivot_table(lens2,index='dept', values =['8 min'], aggfunc='sum', fill_value=0)
cuttoff4 = pd.pivot_table(lens2,index='dept', values =['6 min'], aggfunc='sum', fill_value=0)
cutt = pd.merge(cuttoff1,cuttoff2, on='dept')
cutt = pd.merge(cutt,cuttoff3, on='dept')
cutt = pd.merge(cutt,cuttoff4, on='dept')
cuttlens2 = cutt
cuttlens2

,15 min,12 min,8 min,6 min
dept,,,,
Lens Store,0,1,5,22


## Output

In [109]:

with pd.ExcelWriter(r"C:\Users\user\Documents\Alex\Order Efficiency\November\results\order efficiency results.xlsx", engine='xlsxwriter') as writer:
      controlpivot.to_excel(writer, sheet_name='Control',index=True)
      controldelay.to_excel(writer, sheet_name='Control',index=True, startrow=15)
      cuttcontrol.to_excel(writer, sheet_name='Control', index=True, startrow=9)
      Designerpivot.to_excel(writer, sheet_name='Designer',index=True)
      Designerdelay.to_excel(writer, sheet_name='Designer',index=True, startrow=15)
      cuttdesigner.to_excel(writer, sheet_name='Designer', index=True, startrow=9)
      MainStorepivot.to_excel(writer, sheet_name='Main store',index=True)
      MainStore1delay.to_excel(writer, sheet_name='Main store',index=True, startrow=15)
      cuttMainStore2.to_excel(writer, sheet_name='Main store', index=True, startrow=9)
      Packagingpivot.to_excel(writer, sheet_name='Packaging',index=True)
      Packagingdelay.to_excel(writer, sheet_name='Packaging',index=True, startrow=15)
      cuttpackaging2.to_excel(writer, sheet_name='Packaging', index=True, startrow=9)
      LensStorepivot.to_excel(writer, sheet_name='Lens store',index=True)
      LensStore1delay.to_excel(writer, sheet_name='Lens store', index=True, startrow=15)
      cuttlens2.to_excel(writer, sheet_name='Lens store', index=True, startrow=9)
      criteria_table.to_excel(writer, sheet_name='Criteria', index=True)

def save_xls(list_dfs, xls_path):
    with ExcelWriter(xls_path) as writer:
        for n, df in enumerate(list_dfs):
            df.to_excel(writer,'sheet%s' % n)
        writer.save()